In [4]:
import torch
import numpy as np
from torch import nn

In [5]:


#o = [i + 2*p - k - (k-1)*(d-1)]/s + 1--formula to calculate padding


class Net(nn.Module):

    #YOLO model
    '''
    Input size of the model is 
    448x448x3
    In tensor notation, expressed as [batchsize,3,448,448]
    output--
    [batchsize,30,7,7]
    '''

    def __init__(self):
        super(Net,self).__init__()
        

        self.t1=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=(7,7),stride=2,padding=(2,2)),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
        )
        
        self.t2=nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=(3,3),padding=(1,1)),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
        )
        self.t3=nn.Sequential(
            nn.Conv2d(in_channels=192,out_channels=128,kernel_size=(1,1)),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
        )

        self.t4=nn.Sequential(
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=(3,3),padding=(1,1)),

            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),stride=2)
        )

        self.t5=nn.Sequential(
            nn.Conv2d(in_channels=1024,out_channels=512,kernel_size=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=1024,out_channels=512,kernel_size=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=(3,3),padding=(1,1)),

            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),padding=(1,1)),

            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),stride=2,padding=(1,1))

        )

        self.t6=nn.Sequential(
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),padding=(1,1))
        )


        
    def forward(self,x):
        x=self.t1(x)
        x=self.t2(x)
        x=self.t3(x)
        x=self.t4(x)
        x=self.t5(x)
        x=self.t6(x)
        
        x=torch.flatten(x,1)
        x=nn.Linear(x.size()[1],4096)(x)
        x=nn.Linear(4096,7*7*30)(x)
        x=x.view(-1,30,7,7)

        return x #output of model

In [24]:
device = torch.device('cuda:0')

In [6]:
model = Net()

In [26]:
model.to(device)
model.eval();

In [27]:
dummy_input = torch.randn(1, 3, 448, 448, dtype=torch.float).to(device)
# INIT LOGGERS
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 30
timings=np.zeros((repetitions,1))
#GPU-WARM-UP
for _ in range(10):
    _ = model(dummy_input)
# MEASURE PERFORMANCE
with torch.no_grad():
    for rep in range(repetitions):
        starter.record()
        _ = model(dummy_input)
        ender.record()
        # WAIT FOR GPU SYNC
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings[rep] = curr_time
mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
print(mean_syn)

KeyboardInterrupt: 

In [9]:
%%time
dummy_input = torch.randn(1,3,448,448, dtype=torch.float)
for _ in range(10):
    _ = model(dummy_input)

CPU times: user 25.4 s, sys: 2.89 s, total: 28.3 s
Wall time: 17.1 s
